In [1]:
import numpy as np
A = [[1, 1, 2, 2], [2, 1, 1, 2], [2, 2, 1, 1], [1, 2, 2, 1]]
B = np.array([[1, 1, 2, 2], [2, 2, 1, 1], [2, 2, 1, 1], [1, 2, 2, 1]], dtype=np.int8)
C = np.array([
            [0, 0, 0, 0], 
            [0, 1, 0, 1], 
            [0, 0, 2, 0], 
            [0, 1, 0, 0]], dtype=np.int8)
D = np.array([
            [1, 1, 0, 0], 
            [0, 1, 1, 0], 
            [0, 0, 1, 1], 
            [1, 0, 0, 1]], dtype=np.int8)
E = np.array([  # Unique 
            [1, 2, 2, 1], 
            [2, 1, 1, 2], 
            [2, 0, 0, 0], 
            [1, 2, 0, 0]], dtype=np.int8)
F = np.array([  # Search
            [0, 0, 0, 0], 
            [0, 1, 0, 1], 
            [0, 0, 2, 0], 
            [0, 0, 0, 0]], dtype=np.int8)
# F = np.array([  # Passes validation
#             [2, 1, 1, 2], 
#             [1, 2, 2, 1], 
#             [2, 1, 1, 2], 
#             [1, 2, 2, 1]], dtype=np.int8)


GRID = C.copy()
dim = len(GRID)
dim_2 = dim // 2
FLIP = np.array([0, 2, 1])

In [3]:
A = [1, 2]
A.pop()
A

[1]

In [3]:
def valid_grid(grid, verbose=False):
    """Check that grid is valid."""
    dim = len(grid)
    dim_2 = dim // 2
    
    # Equal count
    if np.any(np.sum(grid == 1, axis=1) > dim_2) or np.any(np.sum(grid == 2, axis=1) > dim_2):
        if verbose:
            print("Unequal count - rows")
        return False
    if np.any(np.sum(grid == 1, axis=0) > dim_2) or np.any(np.sum(grid == 2, axis=0) > dim_2):
        if verbose:
            print("Unequal count - cols")
        return False
    
    # Subsequent (3 in a row)
    for r in range(dim):
        for c in range(dim):
            if c < dim-2 and grid[r, c] == grid[r, c+1] == grid[r, c+2] != 0:
                if verbose:
                    print("Subsequent - rows")
                return False
            if r < dim-2 and grid[r, c] == grid[r+1, c] == grid[r+2, c] != 0:
                if verbose:
                    print("Subsequent - cols")
                return False
    
    # Unique
    full_rows = grid[np.where(grid.all(axis=1))[0]]
    if len(full_rows) != len(np.unique(full_rows, axis=0)):
        if verbose:
            print("Duplicate - rows")
        return False
    
    full_cols = grid.T[np.where(grid.all(axis=0))[0]]
    if len(full_cols) != len(np.unique(full_cols, axis=0)):
        if verbose:
            print("Duplicate - cols")
        return False
    
    return True

## Solver

In [1]:
class Solver():
    """General solver class.
    Helper class to Generator."""

    def solve(self, grid):
        """Returns an exhasuted grid.
        Assumes valid grid."""
        grid = grid.copy()
        while True:
            full_r = np.where(grid.all(axis=1))[0]
            full_c = np.where(grid.all(axis=0))[0]

            if self.subsequent(grid, full_r):
                continue
            if self.subsequent(grid.T, full_c):
                continue
            if self.equal_count(grid, full_r):
                continue
            if self.equal_count(grid.T, full_c):
                continue
            if self.unique(grid, full_r):
                continue
            if self.unique(grid.T, full_c):
                continue
            break

        return grid

    def subsequent(self, rows, full):
        """Checks for subsequent 1s and 2s in rows and cols.
        Returns True if any changes are made."""
        changed = False
        dim = len(rows)

        for r, row in enumerate(rows):
            if r in full:
                continue
            
            for i in range(len(row)-1):
                if row[i] == row[i+1] != 0:
                    if i < dim-2 and rows[r, i+2] == 0: # if next is empty
                        rows[r, i+2] = FLIP[row[i]]
                        changed = True

                    if i > 0 and rows[r, i-1] == 0: # if prev is empty
                        rows[r, i-1] = FLIP[row[i]]
                        changed = True
                    
                if i < len(row)-2 and row[i] == row[i+2] != 0 and row[i+1] == 0:  # Gap between two same
                    rows[r, i+1] = FLIP[row[i]]
                    changed = True

        return changed

    def equal_count(self, rows, full):
        """Checks for equal count of 1s and 2s.
        param rows: 2D array
        param full: set of rows or cols that are already full
        Returns True if any changes are made."""
        changed = False
        dim_2 = len(rows) // 2

        for r, row in enumerate(rows):
            if r in full:
                continue

            if np.sum(row == 1) == dim_2:
                rows[r, np.where(row == 0)[0]] = 2
                changed = True

            elif np.sum(row == 2) == dim_2:
                rows[r, np.where(row == 0)[0]] = 1
                changed = True
        
        return changed

    def unique(self, rows, full):
        """Checks for unique rows and cols.
        Returns True if any changes are made."""
        changed = False

        # Find rows containing exacty two gaps
        gapped_rows_indx = np.where(np.sum(rows == 0, axis=1) == 2)[0]  # [2 3]
        candidates = rows[gapped_rows_indx]  # [[2 1 0 0], [1 0 0 2]]
        full_rows = rows[full]  # [[2 1 1 2]]

        # Compare each candidate with other complete rows
        for i, candy in enumerate(candidates):
            # Find index of colored tiles
            idx = np.where(candy != 0)[0]  # [0 1]
            for row in full_rows:
                if np.array_equal(row[idx], candy[idx]):
                    # Replace the two gaps with complementary colors
                    gaps = np.where(candy == 0)[0]  # [2, 3]
                    rows[gapped_rows_indx[i], gaps] = FLIP[row[gaps]]

                    changed = True
        
        return changed
    
    def get_next(self, grid):
        """Returns the next grid in the sequence.
        Assumes valid grid."""
        grid = grid.copy()

        full_r = np.where(grid.all(axis=1))[0]
        full_c = np.where(grid.all(axis=0))[0]

        if action := self._subsequent(grid, full_r):
            return action
        if action := self._subsequent(grid.T, full_c):
            (y, x), color = action 
            return (x, y), color
        if action := self._equal_count(grid, full_r):
            return action
        if action := self._equal_count(grid.T, full_c):
            (y, x), color = action 
            return (x, y), color
        if action := self._unique(grid, full_r):
            return action
        if action := self._unique(grid.T, full_c):
            (y, x), color = action 
            return (x, y), color
        raise Exception("No action found")
        
    def _subsequent(self, rows, full):
        """Checks for subsequent 1s and 2s in rows and cols.
        Returns True if any changes are made."""
        dim = len(rows)

        for r, row in enumerate(rows):
            if r in full:
                continue
            
            for i in range(len(row)-1):
                if row[i] == row[i+1] != 0:
                    if i < dim-2 and rows[r, i+2] == 0: # if next is empty
                        return (r, i+2), FLIP[row[i]]

                    if i > 0 and rows[r, i-1] == 0: # if prev is empty
                        return (r, i-1), FLIP[row[i]]
                    
                if i < len(row)-2 and row[i] == row[i+2] != 0 and row[i+1] == 0:  # Gap between two same
                    return (r, i+1), FLIP[row[i]]

    def _equal_count(self, rows, full):
        """Checks for equal count of 1s and 2s.
        param rows: 2D array
        param full: set of rows or cols that are already full
        Returns True if any changes are made."""
        dim_2 = len(rows) // 2

        for r, row in enumerate(rows):
            if r in full:
                continue

            if np.sum(row == 1) == dim_2:
                return (r, np.where(row == 0)[0][0]), 2

            elif np.sum(row == 2) == dim_2:
                return (r, np.where(row == 0)[0][0]), 1
        
    def _unique(self, rows, full):
        """Checks for unique rows and cols.
        Returns True if any changes are made."""

        # Find rows containing exacty two gaps
        gapped_rows_indx = np.where(np.sum(rows == 0, axis=1) == 2)[0]  # [2 3]
        candidates = rows[gapped_rows_indx]  # [[2 1 0 0], [1 0 0 2]]
        full_rows = rows[full]  # [[2 1 1 2]]

        # Compare each candidate with other complete rows
        for i, candy in enumerate(candidates):
            # Find index of colored tiles
            idx = np.where(candy != 0)[0]  # [0 1]
            for row in full_rows:
                if np.array_equal(row[idx], candy[idx]):
                    # Replace the two gaps with complementary colors
                    gaps = np.where(candy == 0)[0][0]  # [2, 3]
                    return (gapped_rows_indx[i], gaps), FLIP[row[gaps]]
    

GRID = E.copy()
solver = Solver()
print(GRID)

NameError: name 'E' is not defined

In [59]:
A = np.array([4, 8, 10, 12])
print(480 // A // 12)

[10  5  4  3]


In [52]:
action = solver.get_next(GRID)
print(action)
(y, x), COLOR = action
GRID[y, x] = COLOR
print(GRID)


((3, 3), 2)
[[1 2 2 1]
 [2 1 1 2]
 [2 1 2 1]
 [1 2 1 2]]


In [352]:
solver = Solver()
GRID = np.load('grid.npy')
GRID = solver.solve(GRID)
print(GRID, valid_grid(GRID))

[[1 1 2 2 1 1 2 2]
 [2 2 1 2 1 2 1 1]
 [1 1 2 1 2 1 2 2]
 [2 2 1 1 2 2 1 1]
 [1 1 2 2 1 2 1 2]
 [2 1 1 2 1 1 2 2]
 [2 2 1 1 2 1 2 1]
 [1 2 2 1 2 2 1 1]] True


## Generator

In [349]:
class Generator():
    """Generates Takuzu puzzles."""
    # https://stackoverflow.com/questions/6924216/how-to-generate-sudoku-boards-with-unique-solutions

    def __init__(self):
        self.solver = Solver()
        self.solve = self.solver.solve
    
    def valid_grid(self, grid, verbose=False):
        """Check that grid is valid."""
        dim = len(grid)
        dim_2 = dim // 2
        
        # Equal count
        if np.any(np.sum(grid == 1, axis=1) > dim_2) or np.any(np.sum(grid == 2, axis=1) > dim_2):
            if verbose:
                print("Unequal count - rows")
            return False
        if np.any(np.sum(grid == 1, axis=0) > dim_2) or np.any(np.sum(grid == 2, axis=0) > dim_2):
            if verbose:
                print("Unequal count - cols")
            return False
        
        # Subsequent (3 in a row)
        for r in range(dim):
            for c in range(dim):
                if c < dim-2 and grid[r, c] == grid[r, c+1] == grid[r, c+2] != 0:
                    if verbose:
                        print("Subsequent - rows")
                    return False
                if r < dim-2 and grid[r, c] == grid[r+1, c] == grid[r+2, c] != 0:
                    if verbose:
                        print("Subsequent - cols")
                    return False
        
        # Unique
        full_rows = grid[np.where(grid.all(axis=1))[0]]
        if len(full_rows) != len(np.unique(full_rows, axis=0)):
            if verbose:
                print("Duplicate - rows")
            return False
        
        full_cols = grid.T[np.where(grid.all(axis=0))[0]]
        if len(full_cols) != len(np.unique(full_cols, axis=0)):
            if verbose:
                print("Duplicate - cols")
            return False
        
        return True

    def generate_grid(self, dim):
        print("Generating grid...")
        new = np.zeros((dim, dim), dtype=np.int8)
        complete = self.solve_search(new)
        return complete
        # grid = self.subtract(complete)
        return grid
    
    # Second algorithm from stackoverflow
    def solve_search(self, grid, depth=0):
        """Solves and searches for solution.
        Returns None if no solution found or grid if solution found."""
        # print(depth)
        grid = self.solve(grid)
        
        if not self.valid_grid(grid): # Invalid, so backtrack
            # print(grid,'\n')
            return None
        
        if 0 not in grid:  # Solved, so terminal state
            return grid

        # Randomly select empty cell and fill with 1 or 2
        empty_cells = list(zip(*np.where(grid == 0)))
        tile = empty_cells[np.random.randint(len(empty_cells))]
        color = np.random.randint(1, 3)

        grid[tile] = color
        result = self.solve_search(grid, depth+1)

        if result is None:
            grid[tile] = FLIP[color]
            result = self.solve_search(grid, depth+1)
        return result

    # First algorithm from stackoverflow
    def subtract(self, grid):
        """Subtracts a random tiles until one more subtraction requres solve_search."""
        
        # 2) Shuffle a list of all tiles
        tiles = [(r, c) for r in range(len(grid)) for c in range(len(grid))]
        np.random.shuffle(tiles) 

        while tiles:
            # 3) Remove a tile from grid
            tile = tiles.pop()
            color = grid[tile]
            grid[tile] = 0

            # # 4) Test uniqueness of grid
            # if not self.is_unique(grid):
            #     # 6) If not unique, add tile back and continue
            #     grid[tile] = color

            

        return grid
    
    def is_unique(self, grid):
        """Checks if grid is unique."""
        return self.unique_solution(grid) == 1
    
    def unique_solution(self, grid, solved=False):
        """Solves and searches for solution.
        Returns False if multiple solutions."""

        grid = self.solve(grid)
        
        if not self.valid_grid(grid): # Invalid, so backtrack
            return False
        
        if 0 not in grid:  # Solved, so terminal state
            return 1

        # Find frist instance of 0
        zeros = np.where(grid == 0)
        tile = zeros[0][0], zeros[1][0]

        # First go left, then right
        grid[tile] = 1
        left = self.unique_solution(grid, solved)

        grid[tile] = 2
        right = self.unique_solution(grid, solved)

        if left == 2 or right == 2:
            return 2
        if left == 1 and right == 1:
            return 2
        return self.unique_solution(grid, solved)

generator = Generator()
grid = generator.generate_grid(4)
grid = generator.subtract(grid)
# grid = np.zeros((4, 4), dtype=np.int8)
# grid = np.array([[1, 2], [2, 1]])
print(grid, generator.is_unique(grid))


Generating grid...
[[2 0 2 0]
 [0 0 0 1]
 [0 0 0 0]
 [0 0 2 0]] True


In [339]:
GRID[0, 0] = 
GRID

array([[0, 0, 0, 2],
       [0, 0, 0, 1],
       [2, 1, 2, 1],
       [1, 2, 1, 2]], dtype=int8)

In [249]:
A = np.zeros((4, 4), dtype=np.int8)
A[0, 0] = 1
A[0, 1] = 1
# Find firs tinstance of 0
zeros = np.where(A == 0)
print(zeros)
tile = zeros[0][0], zeros[1][0]
tile

(array([0, 0, 1, 1, 1, 1, 2, 2, 2, 2, 3, 3, 3, 3], dtype=int64), array([2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3], dtype=int64))


(0, 2)

In [201]:
A = [1, 2, 3 ,4]
while A:
    
    A.pop()
    print(A)

[1, 2, 3]
[1, 2]
[1]
[]


## Save and load

In [8]:
# SAVE AND LOAD configirations

# save = {str(i) : a for i, a in enumerate(configs)}  # Give them names 
configs = [[1, 0], A, B]  # Or just use a list
np.savez("duko_configs.npz", *configs)  # Save them

loaded = np.load("duko_configs.npz")
configs = loaded.values()
for kek in configs:
    print(kek)

[1 0]
[[1 1 2 2]
 [2 1 1 2]
 [2 2 1 1]
 [1 2 2 1]]
[[1 1 2 2]
 [2 2 1 1]
 [2 2 1 1]
 [1 2 2 1]]


In [9]:
# np.save('grid.npy', save)  # Save grid

# loaded = np.load('grid.npy')
# loaded